# Analyze Energy Ratios during Baseline Operation

In this notebook, we will demonstrate how to compute and plot the energy ratio between test and reference turbines as a function of wind direction. We'll focus on baseline operation for this example (i.e., without wake steering). The energy ratios can be used to evaluate wake losses experienced by different turbines.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from flasc import floris_tools as ftools
from flasc.dataframe_operations import dataframe_manipulations as dfm
from flasc.energy_ratio import energy_ratio as er
from flasc.energy_ratio.energy_ratio_input import EnergyRatioInput
from flasc.utilities_examples import load_floris_smarteole as load_floris

In [ ]:
# Suppress warnings
import warnings

warnings.filterwarnings("ignore")

# Step 0: Load processed data

Load the processed SCADA data with power curve filtering and northing calibration applied and inspect

In [ ]:
def load_data():
    root_path = Path.cwd()
    f = root_path / "postprocessed" / "df_scada_data_60s_filtered_and_northing_calibrated.ftr"
    df_scada = pd.read_feather(f)

    # # Optionally: downsample to [x] minute averages to speed up things
    # cols_angular = [c for c in df_scada if (("wd_" in c) or ("yaw_" in c))]
    # df_scada = fto.df_downsample(
    #     df_scada,
    #     cols_angular=cols_angular,
    #     window_width=td(seconds=600),
    # )

    return df_scada


df_scada = load_data()

In [ ]:
df_scada.describe()

In [ ]:
df_scada.columns

# Step 1: Limit to baseline data

The SCADA data set contains alternating 1-hour periods with baseline or wake steering control. For these examples, we'll limit the data to baseline operation. 

In [ ]:
df_scada = df_scada[df_scada.control_mode == "baseline"]

# Step 2: Compute reference wind direction, wind speed, and power variables

The energy ratio class as presently implemented requires explicit identification of the dataframe of columns "wd," "ws," and "pow_ref." We'll use the FLORIS model to establish which turbines are unwaked for each wind direction to compute the reference variables.

In [ ]:
# Load FLORIS model of site
fi, turbine_weights = load_floris()

In [ ]:
# Use FLORIS to identify upstream / unwaked turbines for
# each direction
df_upstream = ftools.get_upstream_turbs_floris(fi)

df_upstream.head()

In [ ]:
# Use flasc tools to establish reference wind speeds and directions

# Since will be interested in looking at impacts on SMV5/[4], exclude
# it from each calculation

# Set the wind direction as the average of all turbine averages
df_scada = dfm.set_wd_by_turbines(df_scada, [0, 1, 2, 3, 5, 6])

# Set the wind speed to be the average of all upstream turbines
# (turbines not in a wake in a given direction)
# Except for SMV5
df_scada = dfm.set_ws_by_upstream_turbines(df_scada, df_upstream, exclude_turbs=[4])

# Set the reference power to the average of all upstream turbines
# Except for SMV5
df_scada = dfm.set_pow_ref_by_upstream_turbines(df_scada, df_upstream, exclude_turbs=[4])

# Step 3: Compute and Plot Energy Ratio for Turbine 004

Compare the energy ratio for turbine 004 based on the SCADA data to the equivalent predicted energy ratios from each FLORIS model using our precomputed FLORIS results. Turbine 004 is the downstream waked turbine that benefits from wake steering in the SMARTEOLE wake steering field experiment.

The energy ratios computed here simply represent the ratio between the energy produced by the test turbines and the energy computed using the reference power variable, "pow_ref," as a function of wind direction.

In [ ]:
# Get FLORIS predictions for SCADA dataframe

# Get a list of precalculated FLORIS results
floris_path = Path.cwd() / "precalculated_floris_solutions"
wake_models = ["jensen", "gch", "cc", "turbopark"]
df_fi_list = [None for _ in wake_models]
for wii, wake_model in enumerate(wake_models):
    fn = floris_path / "df_fi_approx_{:s}.ftr".format(wake_model)
    if fn.is_file():
        df_fi_approx = pd.read_feather(fn)
    else:
        raise UserWarning(
            "Please run '01_precalculate_floris_solutions.ipynb' "
            "for the appropriate wake models first."
        )

    df_fi_list[wii] = ftools.interpolate_floris_from_df_approx(
        df=df_scada, df_approx=df_fi_approx, method="linear", verbose=True
    )

In [ ]:
# Set pow_ref in FLORIS results as before
for df_fi in df_fi_list:
    df_fi = dfm.set_pow_ref_by_upstream_turbines(df_fi, df_upstream, exclude_turbs=[4])

In [ ]:
# Calculate and plot energy ratios
er_in = EnergyRatioInput(
    df_fi_list + [df_scada], ["FLORIS: " + wm for wm in wake_models] + ["SCADA data"]
)

In [ ]:
N = 20
print("Calculating energy ratios with bootstrapping (N={}).".format(N))
print("This may take a couple seconds...")
np.random.seed(0)
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    wd_bin_overlap_radius=0.0,
    ws_min=6.0,
    ws_max=12.0,
    N=N,
    percentiles=[5.0, 95.0],
)
ax = er_out.plot_energy_ratios(overlay_frequency=True)
ax[0].set_title("Energy Ratios for Turbine 004")

As shown in the plot above, overall there is good agreement between the SCADA-based energy ratio curve and the energy ratio predictions based on FLORIS results. However, because of relatively little data for many wind directions, the SCADA-based energy ratios are noisy and can deviate from the expected value of 1 when Turbine 004 is unwaked.

# Step 4: Rerun the energy ratio calculation with a different wind speed/wind direction distribution

As an example, we'll create a distribution with uniform frequency across 
all wind speds of interest and concentrated in the direction where SMV6 wakes SMV5.

Can also be used to evaluate the energy ratios under long-term site conditions.

In [ ]:
ws = np.tile(np.arange(6.5, 12.0, 1.0), 180)
wd = np.repeat(np.arange(1.0, 360.0, 2.0), 6)

freq = np.ones_like(ws)
# Increase frequency value in steering wind directions
start_idx = np.where(wd == 169)[0][0]
end_idx = np.where(wd == 231)[0][0]
freq[start_idx:end_idx] = 5
freq = 10 * freq

df_freq = pd.DataFrame({"ws": ws, "wd": wd, "freq_val": freq})

N = 20
print("Calculating energy ratios with bootstrapping (N={}).".format(N))
print("This may take a couple seconds...")
np.random.seed(0)
er_out = er.compute_energy_ratio(
    er_in,
    test_turbines=[4],
    use_predefined_ref=True,
    use_predefined_wd=True,
    use_predefined_ws=True,
    wd_step=2.0,
    wd_bin_overlap_radius=0.0,
    ws_min=6.0,
    ws_max=12.0,
    df_freq=df_freq,
    N=N,
    percentiles=[5.0, 95.0],
)
ax = er_out.plot_energy_ratios(overlay_frequency=True)
ax[0].set_title("Energy Ratios for Turbine 004")